In [1]:
#Import dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs

#Define the path to the chrome driver
def init_browser():
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=True)

#Define a dictionary called scraped_data that will be returned by the function
scraped_data_dict = {}
scraped_data_list = []

#Open a browser window
browser = init_browser()

#Feed the URL and visit the website
url = "https://sofifa.com/?col=vl&sort=desc"
browser.visit(url)

for x in range (1,5):
    #Obtain the HTML and parse it into a BeautifulSoup object
    html = browser.html
    soup = bs(html, "html.parser")
    main_table = soup.find('tbody', class_ = 'list')
    players = main_table.find_all('tr')
    for player in players:
        #Make a dictionary to hold player data
        player_data = {}
        player_data['player_name'] = player.find('div', class_ = 'bp3-text-overflow-ellipsis').get_text()
        #Need to remove the leading space on the name attribute
        player_data['player_primary_position'] = player.find('span', class_ = 'pos').get_text()
        player_data['player_age'] = player.find('td', class_ = 'col col-ae').get_text()
        player_data['player_overall'] = player.find('span', class_ = 'bp3-tag').get_text()
        #under col col-oa
        player_data['player_potential'] = player.find('span', class_ = 'bp3-tag').get_text()
        #under col col-pt
        player_data['player_team'] = player.find('div', class_ = 'bp3-text-overflow-ellipsis').find('a')
        #this overalaps with the player_name, so you would have to search on something else
        scraped_data_list.append(player_data)
    browser.visit("https://sofifa.com/?col=vl&sort=desc"+'&offset='+f'{x*60}')
        #browser.links.find_by_partial_text('Next').click()

browser.quit() 
print(scraped_data_list)

return scraped_data
    

[{'player_name': ' K. Mbappé', 'player_primary_position': 'ST', 'player_age': '21', 'player_overall': '90', 'player_potential': '90', 'player_team': None}, {'player_name': ' Neymar Jr', 'player_primary_position': 'LW', 'player_age': '28', 'player_overall': '91', 'player_potential': '91', 'player_team': None}, {'player_name': ' K. De Bruyne', 'player_primary_position': 'CAM', 'player_age': '29', 'player_overall': '91', 'player_potential': '91', 'player_team': None}, {'player_name': ' J. Sancho', 'player_primary_position': 'RM', 'player_age': '20', 'player_overall': '87', 'player_potential': '87', 'player_team': None}, {'player_name': ' H. Kane', 'player_primary_position': 'ST', 'player_age': '26', 'player_overall': '85', 'player_potential': '85', 'player_team': None}, {'player_name': ' K. Havertz', 'player_primary_position': 'CAM', 'player_age': '21', 'player_overall': '85', 'player_potential': '85', 'player_team': None}, {'player_name': ' S. Mané', 'player_primary_position': 'LW', 'pla